In [9]:
import pandas as pd
import numpy as np
import ewma
import sqrtm_cpp
from portfolio_choice_functions import m_func, m_static

In [2]:
# Definér faste inputparametre
sigma_gam = np.array([[0.05, 0.01, 0.02],
                      [0.01, 0.06, 0.03],
                      [0.02, 0.03, 0.07]])
lambda_mat = np.diag([0.2, 0.5, 0.8])
w = 0.5
mu = 0.05
rf = 0.02
gam = 2
iter = 10

# Kald funktionen m_func
result = m_func(w, mu, rf, sigma_gam, gam, lambda_mat, iter)

# Vis resultatet
print("Resultat matrix:")
print(result)

Resultat matrix:
[[ 0.50214045 -0.03430049 -0.09291139]
 [-0.0137202   0.62137023 -0.07985365]
 [-0.02322785 -0.04990853  0.67206653]]


In [6]:
# Definér faste inputparametre
sigma_gam = np.array([[0.1, 0.02, 0.03],
                      [0.02, 0.15, 0.04],
                      [0.03, 0.04, 0.2]])
lambda_mat = np.diag([0.2, 0.5, 0.8])
w = 0.5
phi = 1.5

# Kald funktionen m_func
result2 = m_static(sigma_gam, w, lambda_mat, phi)

# Vis resultatet
print("Resultat matrix:")
print(result2)

Resultat matrix:
[[ 0.40707902 -0.0566247  -0.09691179]
 [-0.02264988  0.53522722 -0.06757833]
 [-0.02422795 -0.04223646  0.58345104]]


In [12]:
def pf_ts_fun(weights, data, wealth, gam):
    # Første join: Vælg de relevante kolonner fra data og join weights på 'id' og 'eom'
    comb = pd.merge(weights, 
                    data[['id', 'eom', 'ret_ld1', 'pred_ld1', 'lambda']], 
                    on=['id', 'eom'], 
                    how='left')
    
    # Andet join: Join med wealth-data på 'eom'
    comb = pd.merge(comb, 
                    wealth[['eom', 'wealth']], 
                    on='eom', 
                    how='left')
    
    # Gruppér på 'eom' og beregn de ønskede aggregater
    def agg_func(group):
        inv = np.abs(group['w']).sum()
        shorting = np.abs(group.loc[group['w'] < 0, 'w']).sum()
        turnover = np.abs(group['w'] - group['w_start']).sum()
        r_val = (group['w'] * group['ret_ld1']).sum()
        # Antag, at wealth er konstant pr. eom – tag den første værdi
        unique_wealth = group['wealth'].iloc[0]
        tc = unique_wealth / 2 * (group['lambda'] * (group['w'] - group['w_start'])**2).sum()
        return pd.Series({
            'inv': inv,
            'shorting': shorting,
            'turnover': turnover,
            'r': r_val,
            'tc': tc
        })
    
    result = comb.groupby('eom').apply(agg_func).reset_index()
    
    # Beregn eom_ret = eom + 1 måned
    # Her antages det, at eom er af datetime-type; ellers skal du konvertere.
    if np.issubdtype(result['eom'].dtype, np.datetime64):
        result['eom_ret'] = result['eom'] + pd.DateOffset(months=1)
    else:
        # Hvis ikke, kan du evt. blot sætte eom_ret til eom eller konvertere eom til datetime.
        result['eom_ret'] = result['eom']
    
    # Fjern eom-kolonnen
    result = result.drop(columns='eom')
    
    return result

# Eksempeldata:
# Opret et eksempel på weights-DataFrame
weights = pd.DataFrame({
    'id': [1, 1, 2, 2],
    'eom': pd.to_datetime(['2023-01-31', '2023-02-28', '2023-01-31', '2023-02-28']),
    'w': [0.2, -0.1, 0.3, 0.1],
    'w_start': [0.15, -0.05, 0.25, 0.05]
})

# Eksempel på data-DataFrame
data = pd.DataFrame({
    'id': [1, 1, 2, 2],
    'eom': pd.to_datetime(['2023-01-31', '2023-02-28', '2023-01-31', '2023-02-28']),
    'ret_ld1': [0.02, 0.03, -0.01, 0.04],
    'pred_ld1': [0.025, 0.035, -0.005, 0.045],
    'lambda': [0.5, 0.5, 0.8, 0.8]
})

# Eksempel på wealth-DataFrame
wealth = pd.DataFrame({
    'eom': pd.to_datetime(['2023-01-31', '2023-02-28']),
    'wealth': [100000, 105000]
})

# Kald funktionen med et eksempel (gam anvendes her, men virker ikke i udregningen, så vi sender fx 1)
result = pf_ts_fun(weights, data, wealth, gam=1)

print("Resultat:")
print(result)


Resultat:
   inv  shorting  turnover      r       tc    eom_ret
0  0.5       0.0       0.1  0.001  162.500 2023-02-28
1  0.2       0.1       0.1  0.001  170.625 2023-03-28


C:\Users\jonas\AppData\Local\Temp\ipykernel_5624\900762604.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = comb.groupby('eom').apply(agg_func).reset_index()
